# BerlinMOD Queries

So far we have replicated the BerlinMOD Pymeos tutorial using Pyspark. Now we will execute the BerlinMOD queries.

In [1]:
cd "../mobilitydb-berlinmod-sf0.1/"

/data/mobilitydb-berlinmod-sf0.1


/usr/local/lib/python3.9/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
ls

instants.csv*  points.csv*       trips.csv*       vehicles.csv*
licences.csv*  regions.csv*      trips_small.csv  vehicles_small.csv
periods.csv*   spark-warehouse/  vehicle_ids.txt


In [3]:
rm -R spark-warehouse/

## Imports

In [4]:
from pymeos import *
from pymeos.plotters import *

from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pysparkmeos.UDT.MeosDatatype import *
from pysparkmeos.partitions.grid.grid_partitioner import GridPartition
from pysparkmeos.utils.udt_appender import udt_append
from pysparkmeos.utils.utils import *


from pysparkmeos.UDF.udf import *
from pysparkmeos.partitions.mobilityrdd import MobilityRDD


import random, datetime, os, sys
from datetime import timedelta
from functools import partial
from datetime import datetime, timezone
import contextily as cx
import distinctipy
import geopandas as gpd
import pandas as pd
import shapely.geometry as shp

import matplotlib.pyplot as plt
import numpy as np
from shapely import wkb, box, from_wkb
from typing import Union
from time import time

## Spark Initialization

In [5]:
def startspark():
    # Initialize PyMEOS
    pymeos_initialize("UTC")
    
    os.environ['PYSPARK_DRIVER_PYTHON_OPTS']= "notebook"
    os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
    os.environ['PYSPARK_PYTHON'] = sys.executable
    
    #.config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    # Initialize a Spark session
    spark = SparkSession.builder \
        .appName("BerlinMOD with PySpark") \
        .master("local[3]") \
        .config("spark.default.parallelism", 3) \
        .config("spark.executor.memory", "3g") \
        .config("spark.executor.cores", 1) \
        .config("spark.driver.memory", "2g") \
        .config("spark.driver.maxResultSize", 0) \
        .config("spark.sql.allowMultipleTableArguments.enabled", True) \
        .getOrCreate()
    
    # spark.sparkContext.setLogLevel("INFO")
    
    # Append the UDT mapping to the PyMEOS classes
    udt_append()
    
    # Get the value of 'spark.default.parallelism'
    default_parallelism = spark.sparkContext.getConf().get("spark.default.parallelism")
    print(f"spark.default.parallelism: {default_parallelism}")

    # Register udfs in Spark SQL
    register_udfs_under_spark_sql(spark)
    
    return spark

spark = startspark()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/26 12:14:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/26 12:14:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


spark.default.parallelism: 3


## Load Tables
We will use the power of Spark SQL to read in the raw dataframes and then create the tables.

In [6]:
ls

instants.csv*  periods.csv*  regions.csv*  trips_small.csv  vehicles.csv*
licences.csv*  points.csv*   trips.csv*    vehicle_ids.txt  vehicles_small.csv


In [7]:
def load_table(
    spark, 
    path, 
    tablename, 
    partition_key=None, 
    transformation_query=None,
    partition_query = None,
    partitioner_class = None,
    partitioner_args = {},
    **kwargs
):
    print("Reading raw csv ", path)
    rawdf = spark.read.csv(path, **kwargs)

    print("Creating temp view of raw table")
    rawdf.createOrReplaceTempView(f"{tablename}RawNoCache")

    print("Schema and statistics of raw table")
    rawdf.printSchema()
    rawdf.describe().show()
    print(f"Creating final table {tablename} based on {tablename}Raw, partitioned by {partition_key}.")
    spark.sql(f"""DROP TABLE IF EXISTS {tablename}""")

    if transformation_query:
        rawdf = spark.sql(transformation_query)
        rawdf.createOrReplaceTempView(f"{tablename}RawNoCache")
        spark.sql(f"CACHE TABLE {tablename}Raw SELECT * FROM {tablename}RawNoCache")
        spark.sql(f"SELECT * FROM {tablename}Raw LIMIT 5").show()
        #spark.catalog.dropTempView(f"{tablename}RawNoCache")
    else:
        spark.sql(f"CACHE TABLE {tablename}Raw SELECT * FROM {tablename}RawNoCache")
        #spark.catalog.dropTempView(f"{tablename}RawNoCache")
    partitioner = None
    if partition_query:
        bounds = rawdf.rdd.mapPartitions(bounds_calculate_map).reduce(bounds_calculate_reduce)
        print("Bounds: ", bounds)
        partitioner = partitioner_class(bounds=bounds, **partitioner_args)
        grid = partitioner.as_spark_table()
        grid.cache()
        grid.show()
        grid.createOrReplaceTempView("grid")
        print("Creating partitioned table... ")
        partitionedTable = spark.sql(partition_query)
        partitionedTable.createOrReplaceTempView(f"{tablename}Raw")


    start = time()
    
    if partition_key:
        df = spark.sql(f"""
        CREATE TABLE {tablename}NoCache
        USING parquet
        PARTITIONED BY ({partition_key})
        AS SELECT * FROM {tablename}Raw
        """)
    else:
        df = spark.sql(f"""
        CREATE TABLE {tablename}NoCache
        USING parquet
        AS SELECT * FROM {tablename}Raw
        """)
        
    end = time()
    
    if partition_key:
        print(f"{tablename} partitions:")
        spark.sql(f"""
        SHOW PARTITIONS {tablename}NoCache
        """).show()
    print(f"Final table created in {end-start} seconds")

    spark.sql(f"CACHE TABLE {tablename} SELECT * FROM {tablename}NoCache")

    df = spark.table(f"{tablename}")
    
    print(f"Final table {tablename} schema:")
    df.printSchema()
    
    #Drop the temporary view
    #spark.catalog.dropTempView(f"{tablename}Raw")
    return df, (start, end, end-start)


def load_all_tables(configs):
    tables = {}
    stats = {}
    for tablename, config in configs.items():
        table, stat = load_table(**config)
        tables[tablename] = table
        stats[tablename] = stat
    return tables, stats

### Instants

In [8]:
#instants, statsinstants = load_table(spark, "instants.csv", 'instants', inferSchema=True, header=True)
#instants.show()

### Licences

In [9]:
#licences, statslicences = load_table(spark, "licences.csv", 'licences', inferSchema=True, header=True)
#licences.show()

### Periods

In [10]:
transperiod = """
SELECT periodid, beginp, endp, tstzspan(period) AS period FROM periodsRawNoCache
"""
#periods, statsperiods = load_table(spark, "periods.csv", 'periods', transformation_query=transperiod, inferSchema=True, header=True)
#periods.show()

### Points

In [11]:
transpoints = """
SELECT pointid, posx, posy, geometry_from_hexwkb(geom) AS geom FROM pointsRawNoCache
"""
#points, statspoints = load_table(spark, "points.csv", 'points', transformation_query=transpoints, inferSchema=True, header=True)
#points.show()

### Regions

In [12]:
transregions = "SELECT regionid, geometry_from_hexwkb(geom) AS geom FROM regionsRawNoCache"
#regions, statsregions = load_table(spark, "regions.csv", 'regions', transformation_query=transregions, inferSchema=True, header=True)
#regions.show()

### Trips
Note: Use trips_small for testing.

In [13]:
from pyspark.sql.types import Row
from typing import Iterator

schema = StructType([
    StructField("tripid", IntegerType()),
    StructField("vehid", IntegerType()),
    StructField("day", IntegerType()),
    StructField("seqno", IntegerType()),
    StructField("sourcenode", IntegerType()),
    StructField("targetnode", StringType()),
    StructField("trip", TGeomPointSeqSetUDT()),
    StructField("trajectory", GeometryUDT()),
    StructField("license", StringType()),
    StructField("partitionKey", IntegerType())
])

@F.udtf(returnType=schema)
class PartitionUDTF:
    def eval(self, row: Row):
        pymeos_initialize()
        sequence_id = row.tripid
        trajectory = row.trip
        #print(trajectory,type(trajectory))
        #print(row.grid[0])
        #grid = [STBoxWrap(f"SRID=4326;{tile.__str__().strip('SRID=4326;')}") for tile in row.grid]
        grid = row.grid
        gridids = row.gridids
        partitioned = [(key, trajectory.at(tile)) for key, tile in zip(gridids, grid)]
        #print(trajectory)
        #print(grid)
        #print(partitioned)
        count = 0
        responses = []
        for partition_key, partition_traj in partitioned:
            count += 1
            if partition_traj is None:
                continue
            else:
                response = (sequence_id, row.vehid, row.day, row.seqno, row.sourcenode, row.targetnode, partition_traj, row.trajectory, row.licence, partition_key)
                yield response
                #seqs = partition_traj.segments()
                #print(seqs)
                #for partition_traj_seq in seqs:
                #    response = (sequence_id, row.vehid, row.day, row.seqno, row.sourcenode, row.targetnode, partition_traj_seq, row.trajectory, row.licence, partition_key)
                    #responses.append(response)
                    #yield response
        #for response in responses:
        #    yield response


spark.udtf.register("PartitionUDTF", PartitionUDTF)

In [14]:
# We need to add the transformation query to the trips table and inject it to the config.
parttrips = """
    SELECT * 
    FROM PartitionUDTF(
        TABLE(
                SELECT 
                    *, 
                    (SELECT collect_list(tile) FROM grid) AS grid, 
                    (SELECT collect_list(tileid) FROM grid) AS gridids
                FROM tripsRaw
        )
    )
"""

In [15]:

transtrips = "SELECT tripid, vehid, day, seqno, sourcenode, targetnode, trip_from_hexwkb(trip) AS trip, geometry_from_hexwkb(trajectory) AS trajectory, licence FROM tripsRawNoCache"

"""
trips, statstrips = load_table(
    spark, "trips_small.csv", 'trips', 
    transformation_query=transtrips,
    partition_key= 'partitionKey',
    partition_query=parttrips,
    partitioner_class=GridPartition,
    partitioner_args = {'cells_per_side': 3},
    inferSchema=True,
    header=True
)
trips.show()
"""

'\ntrips, statstrips = load_table(\n    spark, "trips_small.csv", \'trips\', \n    transformation_query=transtrips,\n    partition_key= \'partitionKey\',\n    partition_query=parttrips,\n    partitioner_class=GridPartition,\n    partitioner_args = {\'cells_per_side\': 3},\n    inferSchema=True,\n    header=True\n)\ntrips.show()\n'

### Vehicles
Note: Also read vehicles_small for testing.

In [16]:
#vehicles, statsvehicles = load_table(spark, "vehicles_small.csv", 'vehicles', inferSchema=True, header=True)
#vehicles.show()

## Queries

First queries take a general approach and are only used to measure overall performance.

In [17]:
from time import time
def query_exec(query, spark, execute=True, explain=False, explainmode=''):
    plan = None
    if explain:
        plan = spark.sql(f"EXPLAIN {explainmode} {query}").collect()[0].plan
    result = spark.sql(query)
    start = time()
    if execute:
        result.show()
    end = time()
    print("Query execution time: ", end-start, " seconds.")
    return result, (start, end, end-start), plan


def retrieve_exec_stats(queries, starts, ends, durations, plans):
    return pd.DataFrame({"queries": queries, "start": starts, "end": ends, "duration": durations, "plan": plans})


def run_all_queries(queries, spark, execute=True, explain=True, explainmode='', printplan=False):
    """ Utility function to run all queries through subsequent experiments """
    qdfs = []
    starts = []
    ends = []
    durations = []
    plans = []
    for querytext in queries:
        qdf, qstats, plan = query_exec(querytext, spark, execute, explain, explainmode)
        qdfs.append(qdf)
        starts.append(qstats[0])
        ends.append(qstats[1])
        durations.append(qstats[2])
        plans.append(plan)
        if printplan:
            print(plan)
    exec_stats = retrieve_exec_stats(queries, starts, ends, durations, plans)
    return qdfs, exec_stats

### Query 1: What are the models of the vehicles with licence plate numbers from QueryLicences?

In [18]:
querytext1 = """
    SELECT l.licence, v.model
    FROM licences l, vehicles v
    WHERE l.licence = v.licence
"""
#q1, q1stats, plan1 = query_exec(querytext1, spark, explain=True)
#if plan1:
#    print(plan1)

### Query 2: How many vehicles exist that are 'passenger' cars?

In [19]:
querytext2 = """
    SELECT COUNT(licence) AS PassengerCarCount
    FROM vehicles
    WHERE type='passenger'
"""
#q2, q2stats, plan2 = query_exec(querytext2, spark, explain=True)
#if plan2:
#    print(plan2)

### Query 3: Where have the vehicles with licences from QueryLicences1 been at each of the instants from QueryInstants1?

In [20]:
from datetime import datetime

querytext3 = """
    WITH
    veh_w_lic AS (
        SELECT v.vehid, l.licence, v.model
        FROM licences l, vehicles v
        WHERE l.licence = v.licence
    ),
    veh_trips AS (
        SELECT t.* 
        FROM veh_w_lic vw, trips t
        WHERE t.vehid = vw.vehid
    ),
    tile_instants AS (
        SELECT /*+ BROADCAST(gr) */ gr.tile, i.instant
        FROM grid gr, instants i
        WHERE contains_stbox_stbox(gr.tile, i.instant) = TRUE
    )
    SELECT /*+ BROADCAST(i) */ vt.vehid, vt.tripid, vt.trip, i.instant, tpoint_at(vt.trip, i.instant) AS pos
    FROM veh_trips vt, tile_instants i
"""
#q3, q3stats, plan3 = query_exec(querytext3, spark, explain=True)
#if plan3:
#    print(plan3)

### Query 4: Which licence plate numbers belong to vehicles that have passed the points from QueryPoints?

In [21]:
querytext4 = """
    WITH 
    vehids_intersect AS (
        SELECT t.vehid
        FROM trips t, points p
        WHERE ever_touches(t.trip, p.geom) = TRUE
    )
    SELECT vi.vehid, v.licence
    FROM vehids_intersect vi, vehicles v
"""
#q4, q4stats, plan4 = query_exec(querytext4, spark, explain=True)
#if plan4:
#    print(plan4)

In [22]:
dummy_df = spark.createDataFrame(["0"], "string").toDF("temp_clm")
dummy_df.show()

24/05/26 12:14:54 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+--------+
|temp_clm|
+--------+
|       0|
+--------+



In [23]:
queries = [querytext1, querytext2] #, querytext3] #, querytext4]

## Experiments

### Experiment 1: Run Queries ASIS
First we are going to run the queries without any improvement or partitioning strategy.

In [24]:
run_exp_1 = False

#### Create Tables

In [25]:
rm -R spark-warehouse/

/bin/rm: cannot remove 'spark-warehouse/': No such file or directory


In [26]:
configs_exp1 = {
    'instants': {'spark': spark, 'path': 'instants.csv', 'tablename': 'instants', 'inferSchema': True, 'header': True},
    'licences': {'spark': spark, 'path': 'licences.csv', 'tablename': 'licences', 'inferSchema': True, 'header': True},
    'periods':  {'spark': spark, 'path': 'periods.csv', 'tablename': 'periods', 'transformation_query': transperiod, 'inferSchema': True, 'header': True},
    'points':   {'spark': spark, 'path': 'points.csv', 'tablename': 'points', 'transformation_query': transpoints, 'inferSchema': True, 'header': True},
    'regions':  {'spark': spark, 'path': 'regions.csv', 'tablename': 'regions', 'transformation_query':transregions, 'inferSchema': True, 'header': True},
    'trips':    {'spark': spark, 'path': 'trips_small.csv', 'tablename': 'trips', 'transformation_query':transtrips, 'inferSchema': True, 'header': True},
    'vehicles': {'spark': spark, 'path': 'vehicles_small.csv', 'tablename': 'vehicles', 'inferSchema': True, 'header': True}
}

In [27]:
if run_exp_1:
    tables, stats = load_all_tables(configs_exp1)

In [28]:
@F.udtf(returnType="num: int, squared: int")
class SquareNumbers:
    def eval(self, start: int, end: int):
        for num in range(start, end + 1):
            yield (num, num * num)

SquareNumbers(F.lit(1), F.lit(3)).show()

+---+-------+
|num|squared|
+---+-------+
|  1|      1|
|  2|      4|
|  3|      9|
+---+-------+



In [29]:
"""
 |-- tripid: integer (nullable = true)
 |-- vehid: integer (nullable = true)
 |-- day: date (nullable = true)
 |-- seqno: integer (nullable = true)
 |-- sourcenode: integer (nullable = true)
 |-- targetnode: integer (nullable = true)
 |-- trip: pythonuserdefined (nullable = true)
 |-- trajectory: pythonuserdefined (nullable = true)
"""
schema = StructType([
    StructField("point", TGeomPointInstUDT())
])
@F.udtf(returnType=schema)
class ExplodeGeomSeq:
    def eval(self, trip: TGeomPointSeqWrap):
        #print(trip['trip'])
        #trip = trip.trip
        pymeos_initialize()
        instants = trip['trip'].instants()
        for i in instants:
            yield i,

spark.udtf.register("explodeGeomSeq", ExplodeGeomSeq)

In [30]:
if run_exp_1:
    spark.sql("SELECT * FROM explodeGeomSeq(TABLE(SELECT trip FROM trips))").show()

#### Execute Queries

In [31]:
if run_exp_1:
    qdfs_exp1, stats_exp1 = run_all_queries(queries, spark, explain=True, printplan=True)

In [32]:
if run_exp_1:
    for (_id, rdd) in spark.sparkContext._jsc.getPersistentRDDs().items():
        rdd.unpersist()
        print("Unpersisted {} rdd".format(_id))
spark.stop()

### Experiment 2: Partition Trips by vehid, HashPartitioning

In [33]:
run_exp_2 = False

In [34]:
rm -R rm -R spark-warehouse/

/bin/rm: cannot remove 'rm': No such file or directory
/bin/rm: cannot remove 'spark-warehouse/': No such file or directory


In [35]:
if run_exp_2:
    spark = startspark()

#### Create Tables

In [36]:
configs_exp2 = {
    'instants': {'spark': spark, 'path': 'instants.csv', 'tablename': 'instants', 'inferSchema': True, 'header': True},
    'licences': {'spark': spark, 'path': 'licences.csv', 'tablename': 'licences', 'inferSchema': True, 'header': True},
    'periods':  {'spark': spark, 'path': 'periods.csv', 'tablename': 'periods', 'transformation_query': transperiod, 'inferSchema': True, 'header': True},
    'points':   {'spark': spark, 'path': 'points.csv', 'tablename': 'points', 'transformation_query': transpoints, 'inferSchema': True, 'header': True},
    'regions':  {'spark': spark, 'path': 'regions.csv', 'tablename': 'regions', 'transformation_query':transregions, 'inferSchema': True, 'header': True},
    'trips':    {'spark': spark, 'path': 'trips_small.csv', 'tablename': 'trips', 'partition_key': 'vehid', 'transformation_query':transtrips, 'inferSchema': True, 'header': True},
    'vehicles': {'spark': spark, 'path': 'vehicles_small.csv', 'tablename': 'vehicles', 'inferSchema': True, 'header': True}
}

In [37]:
if run_exp_2:
    tables, stats = load_all_tables(configs_exp2)

#### Execute Queries

In [38]:
if run_exp_2:
    qdfs_exp2, stats_exp2 = run_all_queries(queries, spark, explain=True, printplan=True)

In [39]:
#spark.stop()

### Experiment 3: Partition by Trip, RegularGrid

In [40]:
run_exp_3 = True

In [41]:
rm -R rm -R spark-warehouse/

/bin/rm: cannot remove 'rm': No such file or directory
/bin/rm: cannot remove 'spark-warehouse/': No such file or directory


In [42]:
if run_exp_3:
    spark = startspark()

24/05/26 12:15:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


spark.default.parallelism: 3


In [43]:
configs_exp3 = {
    'instants': {'spark': spark, 'path': 'instants.csv', 'tablename': 'instants', 'inferSchema': True, 'header': True},
    'licences': {'spark': spark, 'path': 'licences.csv', 'tablename': 'licences', 'inferSchema': True, 'header': True},
    'periods':  {'spark': spark, 'path': 'periods.csv', 'tablename': 'periods', 'transformation_query': transperiod, 'inferSchema': True, 'header': True},
    'points':   {'spark': spark, 'path': 'points.csv', 'tablename': 'points', 'transformation_query': transpoints, 'inferSchema': True, 'header': True},
    'regions':  {'spark': spark, 'path': 'regions.csv', 'tablename': 'regions', 'transformation_query':transregions, 'inferSchema': True, 'header': True},
    'trips':    {
        'spark': spark, 
        'path': 'trips_small.csv', 
        'tablename': 'trips', 
        'partition_key': 'partitionKey', 
        'transformation_query':transtrips,
        'partition_query': parttrips,
        'partitioner_class': GridPartition,
        'partitioner_args': {'cells_per_side': 3},
        'inferSchema': True, 
        'header': True},
    'vehicles': {'spark': spark, 'path': 'vehicles_small.csv', 'tablename': 'vehicles', 'inferSchema': True, 'header': True}
}

In [44]:
if run_exp_3:
    spark.udtf.register("PartitionUDTF", PartitionUDTF)
    tables, stats = load_all_tables(configs_exp3)

Reading raw csv  instants.csv
Creating temp view of raw table
Schema and statistics of raw table
root
 |-- instantid: integer (nullable = true)
 |-- instant: timestamp (nullable = true)



+-------+------------------+
|summary|         instantid|
+-------+------------------+
|  count|               100|
|   mean|              50.5|
| stddev|29.011491975882016|
|    min|                 1|
|    max|               100|
+-------+------------------+

Creating final table instants based on instantsRaw, partitioned by None.


Final table created in 3.3694961071014404 seconds
Final table instants schema:
root
 |-- instantid: integer (nullable = true)
 |-- instant: timestamp (nullable = true)

Reading raw csv  licences.csv


Creating temp view of raw table
Schema and statistics of raw table
root
 |-- licenceid: integer (nullable = true)
 |-- licence: string (nullable = true)
 |-- vehid: integer (nullable = true)



+-------+------------------+--------+------------------+
|summary|         licenceid| licence|             vehid|
+-------+------------------+--------+------------------+
|  count|               101|     101|               101|
|   mean|              51.0|    NULL|319.46534653465346|
| stddev|29.300170647967224|    NULL| 175.0106604956644|
|    min|                 1|B-BJ 115|                 9|
|    max|               101|B-[U 177|               622|
+-------+------------------+--------+------------------+

Creating final table licences based on licencesRaw, partitioned by None.


...

Bounds:  STBOX XT(((473277.05262936745,6579811.389156611),(498784.34433982597,6606871.682578203)),[2020-06-01 06:01:41.054+00, 2020-06-11 19:30:26.096307+00])
+------+--------------------+
|tileid|                tile|
+------+--------------------+
|     0|STBOX XT(((473277...|
|     1|STBOX XT(((473277...|
|     2|STBOX XT(((473277...|
|     3|STBOX XT(((473277...|
|     4|STBOX XT(((473277...|
|     5|STBOX XT(((473277...|
|     6|STBOX XT(((473277...|
|     7|STBOX XT(((473277...|
|     8|STBOX XT(((473277...|
|     9|STBOX XT(((481779...|
|    10|STBOX XT(((481779...|
|    11|STBOX XT(((481779...|
|    12|STBOX XT(((481779...|
|    13|STBOX XT(((481779...|
|    14|STBOX XT(((481779...|
|    15|STBOX XT(((481779...|
|    16|STBOX XT(((481779...|
|    17|STBOX XT(((481779...|
|    18|STBOX XT(((490281...|
|    19|STBOX XT(((490281...|
+------+--------------------+
only showing top 20 rows

Creating partitioned table... 


trips partitions:
+---------------+
|      partition|
+---------------+
| partitionKey=0|
| partitionKey=1|
|partitionKey=10|
|partitionKey=12|
|partitionKey=13|
|partitionKey=14|
|partitionKey=17|
| partitionKey=2|
|partitionKey=21|
|partitionKey=22|
|partitionKey=23|
|partitionKey=24|
|partitionKey=25|
|partitionKey=26|
| partitionKey=3|
| partitionKey=4|
| partitionKey=5|
| partitionKey=6|
| partitionKey=7|
| partitionKey=8|
+---------------+
only showing top 20 rows

Final table created in 21.399942636489868 seconds


Final table trips schema:
root
 |-- tripid: integer (nullable = true)
 |-- vehid: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- seqno: integer (nullable = true)
 |-- sourcenode: integer (nullable = true)
 |-- targetnode: string (nullable = true)
 |-- trip: pythonuserdefined (nullable = true)
 |-- trajectory: pythonuserdefined (nullable = true)
 |-- license: string (nullable = true)
 |-- partitionKey: integer (nullable = true)

Reading raw csv  vehicles_small.csv
Creating temp view of raw table
Schema and statistics of raw table
root
 |-- vehid: integer (nullable = true)
 |-- licence: string (nullable = true)
 |-- type: string (nullable = true)
 |-- model: string (nullable = true)



+-------+-----------------+-------+-----+--------+
|summary|            vehid|licence| type|   model|
+-------+-----------------+-------+-----+--------+
|  count|                3|      3|    3|       3|
|   mean|             11.0|   NULL| NULL|    NULL|
| stddev|8.717797887081348|   NULL| NULL|    NULL|
|    min|                1|B-CJ 17|  bus|    Opel|
|    max|               17|B-PZ 15|truck|Wartburg|
+-------+-----------------+-------+-----+--------+

Creating final table vehicles based on vehiclesRaw, partitioned by None.


Final table created in 1.659388780593872 seconds
Final table vehicles schema:
root
 |-- vehid: integer (nullable = true)
 |-- licence: string (nullable = true)
 |-- type: string (nullable = true)
 |-- model: string (nullable = true)



In [45]:
spark.sql("SELECT tripid, COUNT(trip) AS cnt FROM trips GROUP BY tripid LIMIT 5").show()

+------+---+
|tripid|cnt|
+------+---+
|   471|  3|
|   481|  3|
|   472|  2|
|    28|  5|
|   436|  1|
+------+---+



In [46]:
trip = spark.sql("SELECT * FROM trips LIMIT 1").collect()[0].trip
instant = spark.sql("SELECT * FROM instants LIMIT 1").collect()[0].instant

print(trip.at(instant))

None


#### Execute Queries

In [47]:
if run_exp_3:
    qdfs_exp3, stats_exp3 = run_all_queries(queries, spark, explain=True, printplan=True)

+-------+-----+
|licence|model|
+-------+-----+
|B-CJ 17| Opel|
+-------+-----+

Query execution time:  0.3990590572357178  seconds.
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [licence#6268, model#6273]
   +- BroadcastHashJoin [licence#6268], [licence#6271], Inner, BuildRight, false
      :- Filter isnotnull(licence#6268)
      :  +- Scan In-memory table licences [licence#6268], [isnotnull(licence#6268)]
      :        +- InMemoryRelation [licenceid#6267, licence#6268, vehid#6269], StorageLevel(disk, memory, deserialized, 1 replicas)
      :              +- *(1) ColumnarToRow
      :                 +- FileScan parquet spark_catalog.default.licencesnocache[licenceid#823,licence#824,vehid#825] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/data/mobilitydb-berlinmod-sf0.1/spark-warehouse/licencesnocache], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<licenceid:int,licence:string,vehid:int>
      +- Broadca

In [55]:
%%time
querytext3 = """
    WITH
    veh_w_lic AS (
        SELECT v.vehid, l.licence, v.model
        FROM licences l, vehicles v
        WHERE l.licence = v.licence
    ),
    veh_trips AS (
        SELECT t.* 
        FROM veh_w_lic vw, trips t
        WHERE t.vehid = vw.vehid
    ),
    tile_instants AS (
        SELECT /*+ BROADCAST(gr) */ gr.tile, i.instant
        FROM grid gr, instants i
        WHERE temporally_overlaps(datetime_to_tinstant(i.instant), gr.tile) = TRUE
    )
    SELECT /*+ BROADCAST(i) */ vt.vehid, vt.tripid, vt.trip, i.instant, tpoint_at(vt.trip, i.instant) AS pos
    FROM veh_trips vt, tile_instants i
    WHERE tpoint_at(vt.trip, i.instant) IS NOT NULL
"""

q3 = spark.sql(querytext3)
q3.show()

#def delete_nulls(partition):
#    for row in partition:
#        if row.pos != None:
#            yield row
    
#q3.rdd.mapPartitions(delete_nulls).take(5)

24/05/26 12:50:53 WARN ExtractPythonUDFFromJoinCondition: The join condition:temporally_overlaps(datetime_to_tinstant(instant#8134)#8135, tile#4306)#8136 of the join plan contains PythonUDF only, it will be moved out and the join plan will be turned to cross join.
24/05/26 12:50:53 WARN ExtractPythonUDFFromJoinCondition: The join condition:isnotnull(tpoint_at(trip#8129, instant#8134)#8137) of the join plan contains PythonUDF only, it will be moved out and the join plan will be turned to cross join.
24/05/26 13:00:34 ERROR PythonUDFRunner: Python worker exited unexpectedly (crashed)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1225, in main
    eval_type = read_int(infile)
  File "/usr/local/lib/python3.9/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 596, in read_int
    raise EOFError
EOFError

	at org.apache.spark.api.py

Py4JJavaError: An error occurred while calling o736.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 141.0 failed 1 times, most recent failure: Lost task 0.0 in stage 141.0 (TID 136) (336d95f83d14 executor driver): java.net.SocketException: Connection reset
	at java.base/sun.nio.ch.NioSocketImpl.implRead(NioSocketImpl.java:318)
	at java.base/sun.nio.ch.NioSocketImpl.read(NioSocketImpl.java:346)
	at java.base/sun.nio.ch.NioSocketImpl$1.read(NioSocketImpl.java:796)
	at java.base/java.net.Socket$SocketInputStream.read(Socket.java:1099)
	at java.base/java.io.BufferedInputStream.fill(BufferedInputStream.java:291)
	at java.base/java.io.BufferedInputStream.read1(BufferedInputStream.java:347)
	at java.base/java.io.BufferedInputStream.implRead(BufferedInputStream.java:420)
	at java.base/java.io.BufferedInputStream.read(BufferedInputStream.java:399)
	at java.base/java.io.DataInputStream.readFully(DataInputStream.java:208)
	at java.base/java.io.Data

24/05/26 13:00:39 WARN PythonUDFRunner: Incomplete task 2.0 in stage 141 (TID 137) interrupted: Attempting to kill Python Worker
24/05/26 13:00:39 WARN PythonUDFRunner: Incomplete task 2.0 in stage 141 (TID 137) interrupted: Attempting to kill Python Worker
24/05/26 13:00:39 WARN TaskSetManager: Lost task 1.0 in stage 141.0 (TID 137) (336d95f83d14 executor driver): TaskKilled (Stage cancelled: Job aborted due to stage failure: Task 0 in stage 141.0 failed 1 times, most recent failure: Lost task 0.0 in stage 141.0 (TID 136) (336d95f83d14 executor driver): java.net.SocketException: Connection reset
	at java.base/sun.nio.ch.NioSocketImpl.implRead(NioSocketImpl.java:318)
	at java.base/sun.nio.ch.NioSocketImpl.read(NioSocketImpl.java:346)
	at java.base/sun.nio.ch.NioSocketImpl$1.read(NioSocketImpl.java:796)
	at java.base/java.net.Socket$SocketInputStream.read(Socket.java:1099)
	at java.base/java.io.BufferedInputStream.fill(BufferedInputStream.java:291)
	at java.base/java.io.BufferedInputStr

In [48]:
%%time
@F.udf(returnType=BooleanType())
def contains_stbox_stbox(stbox, other):
    pymeos_initialize()
    return stbox.contains(other)
spark.udf.register("contains_stbox_stbox", contains_stbox_stbox)

@F.udf(returnType=BooleanType())
def temporally_overlaps(temporal, other):
    pymeos_initialize()
    return temporal.temporally_overlaps(other)
spark.udf.register("temporally_overlaps", temporally_overlaps)

@F.udf(returnType=TBoolInstUDT())
def datetime_to_tinstant(instant):
    pymeos_initialize()
    return TBoolInst.from_base_time(value=True, base=instant)
spark.udf.register("datetime_to_tinstant", datetime_to_tinstant)

CPU times: user 58 ms, sys: 17.5 ms, total: 75.5 ms
Wall time: 305 ms


In [78]:
trips481 = spark.sql("SELECT * FROM trips").collect()
instantss = spark.sql("SELECT * FROM instants").collect()
gridd = spark.sql("SELECT * FROM grid").collect()

In [86]:
for j, instantrow in enumerate(instantss):
    inst = instantrow.instant
    for k, triprow in enumerate(trips481):
        tp = triprow.trip
        tpid = triprow.tripid
        if tp.temporally_contains(inst):
            #print(tile.tmin(), tile.tmax(), inst, tile.contains(inst))
            #print(tpid, tp.at(inst))
            for i, griddrow in enumerate(gridd):
                tile = griddrow.tile
                tileid = griddrow.tileid
                if tile.overlaps(inst):
                    print(tile.contains(inst), tpid, tileid, tp.at(inst))
        #if k == 2: break
    #if j == 1: break
#if i == 1: break

True 3 0 POINT(486076.9615656639 6593385.494335708)@2020-06-02 07:23:47+00
True 3 3 POINT(486076.9615656639 6593385.494335708)@2020-06-02 07:23:47+00
True 3 6 POINT(486076.9615656639 6593385.494335708)@2020-06-02 07:23:47+00
True 3 9 POINT(486076.9615656639 6593385.494335708)@2020-06-02 07:23:47+00
True 3 12 POINT(486076.9615656639 6593385.494335708)@2020-06-02 07:23:47+00
True 3 15 POINT(486076.9615656639 6593385.494335708)@2020-06-02 07:23:47+00
True 3 18 POINT(486076.9615656639 6593385.494335708)@2020-06-02 07:23:47+00
True 3 21 POINT(486076.9615656639 6593385.494335708)@2020-06-02 07:23:47+00
True 3 24 POINT(486076.9615656639 6593385.494335708)@2020-06-02 07:23:47+00
True 468 0 POINT(474047.34709978715 6600073.318555428)@2020-06-02 07:23:47+00
True 468 3 POINT(474047.34709978715 6600073.318555428)@2020-06-02 07:23:47+00
True 468 6 POINT(474047.34709978715 6600073.318555428)@2020-06-02 07:23:47+00
True 468 9 POINT(474047.34709978715 6600073.318555428)@2020-06-02 07:23:47+00
True 468

In [54]:
%%time
spark.sql("""
        SELECT /*+ BROADCAST(gr) */ COUNT(*)
        FROM grid gr, instants i
        WHERE temporally_overlaps(datetime_to_tinstant(i.instant), gr.tile) = TRUE
""").show(truncate=False)

24/05/26 12:44:43 WARN ExtractPythonUDFFromJoinCondition: The join condition:temporally_overlaps(datetime_to_tinstant(instant#7990)#7991, tile#4306)#7993 of the join plan contains PythonUDF only, it will be moved out and the join plan will be turned to cross join.


+--------+
|count(1)|
+--------+
|837     |
+--------+

CPU times: user 240 ms, sys: 18.5 ms, total: 259 ms
Wall time: 1min 27s


In [50]:
%%time

spark.sql("SELECT datetime_to_tinstant(instant) FROM instants LIMIT 1").show()

+-----------------------------+
|datetime_to_tinstant(instant)|
+-----------------------------+
|         t@2020-06-01 19:4...|
+-----------------------------+

CPU times: user 17.6 ms, sys: 10.6 ms, total: 28.2 ms
Wall time: 1.18 s
